## CSV dosyasındaki verinin yüklenmesini gerçekleştirme

In [1]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "IMDb_Top_1000_Movies_Dataset.csv"
loader = CSVLoader(filepath, encoding="utf-8")

data = loader.load()

print(f"Toplam {len(data)} film yüklendi.")

c:\Users\Nihal\miniconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Toplam 1000 film yüklendi.


## Veriyi parçalarına ayırma(Chunking işlemi)

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

print(f"Number of documents after splitting: {len(docs)}")
docs[7]

Number of documents after splitting: 1245


Document(metadata={'source': 'IMDb_Top_1000_Movies_Dataset.csv', 'row': 6}, page_content=": 6\nMovie_Name: Schindler's List\nMovie_Rating: 9.0\nMovie_Certificate: A\nMovie_Year: 1993\nMovie_Runtime: 195 min\nMovie_Cast: Liam Neeson, Ralph Fiennes, Ben Kingsley, Caroline Goodall\nMovie_Genre: Biography, Drama, History\nMovie_Poster: https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_UX67_CR0,0,67,98_AL_.jpg\nMovie_Description: In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis.\nMovie_Poster_HD: https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg")

## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [4]:
load_dotenv()  # Load environment variables from .env file
import os
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    print("HATA: API anahtarı .env dosyasından okunamadı!")
else:
    print(f"Anahtar başarıyla yüklendi. İlk 4 karakter: {api_key[:4]}")

Anahtar başarıyla yüklendi. İlk 4 karakter: AIza


# Huggingface alternatifi

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector = embeddings.embed_query("hello, world!")
vector[:5]


In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

GoogleGenerativeAIError: Error embedding content (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 1000, model: gemini-embedding-1.0\nPlease retry in 30.227870969s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/embed_content_free_tier_requests', 'quotaId': 'EmbedContentRequestsPerDayPerProjectPerModel-FreeTier',

## ChromaDB üzerine kayıt işlemi

In [ ]:
from langchain_chroma import Chroma

In [ ]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5 })

In [ ]:
retrieved_docs = retriever.invoke("high budget expensive movies")     

len(retrieved_docs)

print(retrieved_docs[5].page_content) 


## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [ ]:
system prompt = (
    "You are assistant for question-answering tasks."
    "Use the following context to answer the question at the end."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "\n\n"
    "{context}"
)



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [ ]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)


## RAG Zinciri Oluşturma (RAG + LLM)


In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

## Kullanıcı sorgusunu çalıştırma

In [ ]:
response = rag_chain.invoke({"input": "Explain the transformer architecture?"})

print(response["answer"])